In [1]:
from configparser import ConfigParser
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
from numpy import matrix
import pickle
from sklearn.metrics import mean_squared_error
from get_timbre import get_timbre
from evaluation import evaluate_data
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

parser = ConfigParser()
parser.read('./spotify_credentials.cfg')

SPOTIPY_CLIENT_ID = parser.get('spotify', 'SPOTIPY_CLIENT_ID')
SPOTIPY_CLIENT_SECRET = parser.get('spotify', 'SPOTIPY_CLIENT_SECRET')

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET))

user_id = 'czdoifmfngjhvoetavlok9dg5'

In [14]:
playlist_ids = ['5odVaQ10ISGhvuakjVGmxp', '0mCV9EHiVXEsbw307lBF25', '4VGvI1dODxGyBop5jFhgfN',
                '5pVRL48wvxNRL8lWkxdVCX', '5Kq61WYIbYGLB6dvuTKzZV', '3MO4sQXwvB13TAdiSoG9AR',
                '64iSau8h74DkOUG8JVY3W2', '5ZWw4qko5ccxnEIA8blVSq', '35iVxmwsHyJVSdvRasuK8V',
                '06LB9aYKvibLEXb4Qf3gHM', '2dfxSJXVFYpZi2YOjgbwRX', '6FwDZbpBnGXBOJ1UwW9lKb',
                '44vN1eprBxUdirtb9pCOge', '64mSbvFQYIW547z4zWrEQK', '3VBWC197hTEmayqbDTmvp4',
                '2Am8cD0icf62d4dL7NrDTx', '0fjjY976NVJ03tPhxjamcX', '0vzdOGXJBf6SBFeYbwfpnL',
                '5JufAIzUR25lgE8zrums4k', '2ppGD3RwccxuPSJZiyZrCm', '4LBHOUvGlk4RtaMbfB5huK',
                '1VJZKBsulYCry6QpSoW5mG', '0dnsYgOOtfsOg78qLrZEAv', '5KtPLkVfM18fVQ4puapK98',
                '51eXnQHgNcFw7xGllU5kCf', '23x3CI4Cncf6BpSnIeZlF7', '1rDhjtHJkAIlUWgJlxQSRF',
                '5cS1vW9gzr2lLFZ0XpkmfD', '2xxXRaeMZSoAiUslTqbB7S', '0Gx1JfJVTkWSfCcTkMgJiA',
                ]

# playlist_ids = ['5odVaQ10ISGhvuakjVGmxp']

playlist_ids_random = ['a','b','c','d']

In [3]:
# Get all track data (has been run, no longer needed)

# x = 1
# for playlist_id in playlist_ids:
#     print('Running playlist ' + playlist_id)

#     # Get track data
    
#     results = sp.playlist_tracks(playlist_id)
#     tracks = results['items']
#     track_ids = []
#     for track in tracks:
#         track_ids.append(track['track']['id'])

#     columns_to_remove = ['analysis_url', 'type', 'uri', 'track_href']
#     headings = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
#                 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
#                 'id', 'duration_ms', 'time_signature', 'track_title', 'album_title',
#                 'album_artist', 'track_number', 'total_tracks']

#     playlist_data = pd.DataFrame(columns=headings)
#     playlist_length = len(track_ids)

#     i = 0
#     for track in track_ids:
#         results = sp.audio_features(track)
#         if results[0]: 
#             features = results[0]
        
#         features_matrix = pd.DataFrame.from_records(features, index=[0])
        
#         # Remove unneeded columns
#         features_matrix.drop(columns = columns_to_remove, axis = 1, inplace = True)

#         # Add track info
#         features_matrix['track_title'] = tracks[i]['track']['name']
#         features_matrix['album_title'] = tracks[i]['track']['album']['name']
#         features_matrix['album_artist'] = tracks[i]['track']['album']['artists'][0]['name']

#         # Add track number and total tracks
#         features_matrix['track_number'] = tracks[i]['track']['track_number']
#         features_matrix['total_tracks'] = playlist_length

#         playlist_data = pd.concat([playlist_data, features_matrix])
#         i += 1

#     playlist_data = playlist_data.reset_index(drop=True)
#     playlist_data_full = pd.DataFrame(playlist_data)

#     # Get timbre data

#     timbre = pd.DataFrame(columns = ['song_timbre', 'song_timbre_start', 'song_timbre_end', 'loudness_start', 'loudness_end'])

#     for track in track_ids:
#         print('Getting timbre for: '+ track)
#         track_timbre = get_timbre(track, playlist_data_full)
#         timbre = pd.concat([timbre, track_timbre], axis=0)

#     timbre = timbre.reset_index(drop=True)
#     playlist_data = pd.concat([playlist_data, timbre], axis=1)

#     playlist_data.to_csv('./data/test_set/playlist_data_' + str(x) + '.csv')
#     playlist_data_full.to_csv('./data/test_set/playlist_data_full_' + str(x) + '.csv')
#     x += 1

In [ ]:
# Evalute all tracks in album playlists

x = 1
sorted_evals = []
original_evals = []
mean_squared_errors = []
eval_changes = []

for playlist_id in playlist_ids:
    
    print('Running playlist ' + playlist_id)

    # Read and set up data
    playlist_data = pd.read_csv('./data/test_set/playlist_data_' + str(x) + '.csv')
    playlist_data.drop(columns = ['Unnamed: 0'], axis = 1, inplace = True)
    playlist_data_full = playlist_data.copy(deep=True)
    x += 1

    # Remove unnecessary testing columns
    columns_to_remove = ['id', 'track_title', 'album_title', 'album_artist',
                        'track_number', 'total_tracks', 'key', 'mode',
                        'duration_ms', 'time_signature']
    playlist_data.drop(columns = columns_to_remove, axis = 1, inplace = True)

    # Evaluate original order
    original_order_eval = evaluate_data(playlist_data_full)
    original_evals.append(original_order_eval)

    # Load and run trained model
    model = pickle.load(open('./regression/random_forest.sav', 'rb')) # change to whichever model we want to use
    playlist_data_full['order'] = model.predict(playlist_data.values)
    playlist_data_full = playlist_data_full.sort_values(by=['order'])
    playlist_data_full['order'] = np.arange(1, playlist_data_full.shape[0]+1) # convert order to integer playlist track number

    # Evaluate new order
    sorted_order_eval = evaluate_data(playlist_data_full)
    sorted_evals.append(sorted_order_eval)

    y_true = playlist_data_full['track_number']
    y_pred = playlist_data_full['order']
    mean_squared_errors.append(mean_squared_error(y_true, y_pred))

    track_eval_changes = []
    
    for metric in range(len(original_order_eval)):
        track_eval_changes.append(sorted_order_eval[metric] - original_order_eval[metric])
        
    eval_changes.append(track_eval_changes)

In [ ]:
# Evalute all tracks in random playlists

x = 1
sorted_evals_random = []
original_evals_random = []
mean_squared_errors_random = []
eval_changes_random = []

for playlist_id in playlist_ids_random:
    
    print('Running playlist ' + playlist_id)

    # Read and set up data
    playlist_data = pd.read_csv('./data/test_set/playlist_data_random_' + str(x) + '.csv')
    playlist_data.drop(columns = ['Unnamed: 0'], axis = 1, inplace = True)
    playlist_data_full = playlist_data.copy(deep=True)
    x += 1

    # Remove unnecessary testing columns
    columns_to_remove = ['id', 'track_title', 'album_title', 'album_artist',
                        'track_number', 'total_tracks', 'key', 'mode',
                        'duration_ms', 'time_signature']
    playlist_data.drop(columns = columns_to_remove, axis = 1, inplace = True)

    # Evaluate original order
    original_order_eval = evaluate_data(playlist_data_full)
    original_evals_random.append(original_order_eval)

    # Load and run trained model
    model = pickle.load(open('./regression/random_forest.sav', 'rb')) # change to whichever model we want to use
    playlist_data_full['order'] = model.predict(playlist_data.values)
    playlist_data_full = playlist_data_full.sort_values(by=['order'])
    playlist_data_full['order'] = np.arange(1, playlist_data_full.shape[0]+1) # convert order to integer playlist track number

    # Evaluate new order
    sorted_order_eval = evaluate_data(playlist_data_full)
    sorted_evals_random.append(sorted_order_eval)

    y_true = playlist_data_full['track_number']
    y_pred = playlist_data_full['order']
    mean_squared_errors_random.append(mean_squared_error(y_true, y_pred))

    track_eval_changes = []
    
    for metric in range(len(original_order_eval)):
        track_eval_changes.append(sorted_order_eval[metric] - original_order_eval[metric])
        
    eval_changes_random.append(track_eval_changes)

In [16]:
print(np.mean(mean_squared_errors), np.mean(mean_squared_errors_random))

evaluation_vector_titles = ['keyTransition', 'tempoTransition', 'trackNumTransition', 'loudenssTransition', 'timbreTransition',
                            'danceabilityTransition', 'energyTransition', 'speechinessTransition','acousticnessTransition',
                            'instrumentalnessTransition', 'livenessTransition', 'valenceTransition']
original_evals_df = pd.DataFrame.from_records(original_evals, columns=evaluation_vector_titles)
sorted_evals_df = pd.DataFrame.from_records(sorted_evals, columns=evaluation_vector_titles)
eval_changes_df = pd.DataFrame.from_records(eval_changes, columns=evaluation_vector_titles)
original_evals_random_df = pd.DataFrame.from_records(original_evals_random, columns=evaluation_vector_titles)
sorted_evals_random_df = pd.DataFrame.from_records(sorted_evals_random, columns=evaluation_vector_titles)
eval_changes_random_df = pd.DataFrame.from_records(eval_changes_random, columns=evaluation_vector_titles)

original_evals_df['trackTotal'] = original_evals_df.mean(numeric_only=True, axis=1)
original_evals_df.loc['metricTotalAlbumOriginal'] = original_evals_df.mean(numeric_only=True, axis=0)
sorted_evals_df['trackTotal'] = sorted_evals_df.mean(numeric_only=True, axis=1)
sorted_evals_df.loc['metricTotalAlbumSorted'] = sorted_evals_df.mean(numeric_only=True, axis=0)
eval_changes_df['trackTotal'] = eval_changes_df.mean(numeric_only=True, axis=1)
eval_changes_df.loc['metricTotalAlbumChanges'] = eval_changes_df.mean(numeric_only=True, axis=0)
original_evals_random_df['trackTotal'] = original_evals_random_df.mean(numeric_only=True, axis=1)
original_evals_random_df.loc['metricTotalRandomOriginal'] = original_evals_random_df.mean(numeric_only=True, axis=0)
sorted_evals_random_df['trackTotal'] = sorted_evals_random_df.mean(numeric_only=True, axis=1)
sorted_evals_random_df.loc['metricTotalRandomSorted'] = sorted_evals_random_df.mean(numeric_only=True, axis=0)
eval_changes_random_df['trackTotal'] = eval_changes_random_df.mean(numeric_only=True, axis=1)
eval_changes_random_df.loc['metricTotalRandomChanges'] = eval_changes_random_df.mean(numeric_only=True, axis=0)

evals_df = pd.DataFrame([original_evals_df.loc['metricTotalAlbumOriginal'],
                        sorted_evals_df.loc['metricTotalAlbumSorted'],
                        eval_changes_df.loc['metricTotalAlbumChanges'],
                        original_evals_random_df.loc['metricTotalRandomOriginal'],
                        sorted_evals_random_df.loc['metricTotalRandomSorted'],
                        eval_changes_random_df.loc['metricTotalRandomChanges']])

evals_df

68.5236669506036 2070.697327586207


,keyTransition,tempoTransition,trackNumTransition,loudenssTransition,timbreTransition,danceabilityTransition,energyTransition,speechinessTransition,acousticnessTransition,instrumentalnessTransition,livenessTransition,valenceTransition,trackTotal
metricTotalAlbumOriginal,0.307445,0.270573,0.962101,0.0,0.780072,0.282501,0.195262,0.199074,0.200991,0.299489,0.151445,0.101562,0.312543
metricTotalAlbumSorted,0.268975,0.295807,0.072249,0.0,0.751015,0.259208,0.230544,0.214551,0.186966,0.306691,0.149131,0.140875,0.239668
metricTotalAlbumChanges,-0.038470,0.025234,-0.889852,0.0,-0.029057,-0.023292,0.035282,0.015477,-0.014025,0.007202,-0.002314,0.039313,-0.072875
metricTotalRandomOriginal,0.291990,0.337591,0.102684,0.0,0.732499,0.219139,0.337444,0.084244,0.059226,0.261687,0.055702,0.232294,0.226208
metricTotalRandomSorted,0.338208,0.308903,0.125793,0.0,0.710770,0.250969,0.302854,0.091819,0.020965,0.242630,0.042694,0.156624,0.216019
metricTotalRandomChanges,0.046218,-0.028688,0.023109,0.0,-0.021729,0.031830,-0.034590,0.007576,-0.038261,-0.019057,-0.013008,-0.075669,-0.010189


In [ ]:
# Evalute all tracks with autoencoder

encoder = tf.keras.models.load_model('./regression/encoder.h5')

x = 1
sorted_evals = []
original_evals = []
mean_squared_errors = []
eval_changes = []

for playlist_id in playlist_ids:
    
    print('Running playlist ' + playlist_id)

    # Read and set up data
    playlist_data = pd.read_csv('./data/test_set/playlist_data_' + str(x) + '.csv')
    playlist_data.drop(columns = ['Unnamed: 0'], axis = 1, inplace = True)
    playlist_data_full = playlist_data.copy(deep=True)
    x += 1

    # Remove unnecessary testing columns
    columns_to_remove = ['id', 'track_title', 'album_title', 'album_artist',
                        'track_number', 'total_tracks', 'key', 'mode',
                        'duration_ms', 'time_signature']
    playlist_data.drop(columns = columns_to_remove, axis = 1, inplace = True)

    # Evaluate original order
    original_order_eval = evaluate_data(playlist_data_full)
    original_evals.append(original_order_eval)

    # Load and run trained model
    encoded_playlist_data = encoder.predict(playlist_data)
    model = pickle.load(open('./regression/autoencoder_random_forest.sav', 'rb')) # change to whichever model we want to use
    playlist_data_full['order'] = model.predict(encoded_playlist_data)
    playlist_data_full = playlist_data_full.sort_values(by=['order'])
    playlist_data_full['order'] = np.arange(1, playlist_data_full.shape[0]+1) # convert order to integer playlist track number

    # Evaluate new order
    sorted_order_eval = evaluate_data(playlist_data_full)
    sorted_evals.append(sorted_order_eval)

    y_true = playlist_data_full['track_number']
    y_pred = playlist_data_full['order']
    mean_squared_errors.append(mean_squared_error(y_true, y_pred))

    track_eval_changes = []
    
    for metric in range(len(original_order_eval)):
        track_eval_changes.append(sorted_order_eval[metric] - original_order_eval[metric])
        
    eval_changes.append(track_eval_changes)

In [ ]:
print(np.mean(mean_squared_errors))

evaluation_vector_titles = ['keyTransition', 'tempoTransition', 'trackNumTransition', 'loudenssTransition', 'timbreTransition',
                            'danceabilityTransition', 'energyTransition', 'speechinessTransition','acousticnessTransition',
                            'instrumentalnessTransition', 'livenessTransition', 'valenceTransition']
eval_changes_df = pd.DataFrame.from_records(eval_changes, columns=evaluation_vector_titles)

eval_changes_df['trackTotal'] = eval_changes_df.abs().mean(numeric_only=True, axis=1)
eval_changes_df.loc['metricTotal'] = eval_changes_df.abs().mean(numeric_only=True, axis=0)

eval_changes_df

In [ ]:
# # get permissions to rearrange
# from spotipy.oauth2 import SpotifyOAuth
# scope = "playlist-modify-public"
# sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope,redirect_uri='http://localhost:5678/',client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET))

# # rearrange
# sorted_ids = list(playlist_data_full['id'])

# UPDATED_PLAYLIST = sp.playlist_replace_items(playlist_id,sorted_ids)
# UPDATED_PLAYLIST

In [ ]:
# Predict Next Random Forest

x = 1
sorted_evals_next = []
original_evals_next = []
mean_squared_errors = []
eval_changes_next = []

for playlist_id in playlist_ids_random:
    
    print('Running playlist ' + playlist_id)

    # Read and set up data
    playlist_data = pd.read_csv('./data/test_set/playlist_data_random_' + str(x) + '.csv')
    playlist_data.drop(columns = ['Unnamed: 0'], axis = 1, inplace = True)
    playlist_data_full = playlist_data.copy(deep=True)
    playlist_data.loc[:,['loudness', 'tempo', 'loudness_start', 'loudness_end']] = MinMaxScaler().fit_transform(playlist_data.loc[:,['loudness', 'tempo', 'loudness_start', 'loudness_end']])
    x += 1

    # Remove unnecessary testing columns
    columns_to_remove = ['id', 'track_title', 'album_title', 'album_artist',
                        'track_number', 'total_tracks', 'key', 'mode',
                        'duration_ms', 'time_signature']
    playlist_data.drop(columns = columns_to_remove, axis = 1, inplace = True)

    # Evaluate original order
    original_order_eval = evaluate_data(playlist_data_full)
    original_evals_next.append(original_order_eval)

    # Load and run trained model
    model = pickle.load(open('./regression/rf_predict_next.sav', 'rb')) # change to whichever model we want to use
    optimal_songs = model.predict(playlist_data.values)
    
    # Order songs based on optimal result
    sorted_order = [0]
    for current_track in range(playlist_data.shape[0]-1):
        distances = []
        for other_track in range(playlist_data.shape[0]):
            # print(other_track)
            if current_track != other_track:
                distances.append(np.linalg.norm(optimal_songs[current_track]-playlist_data.iloc[other_track]))
            else:
                distances.append(1)
        distances_temp = distances.copy()
        # print('D ' + str(distances))
        for other_track in range(len(distances)):
            # print('OT ' + str(other_track))
            # print('D_Temp ' + str(distances_temp))
            if distances.index(min(distances_temp)) not in sorted_order:
                # print('SO ' + str(distances.index(min(distances_temp))))
                sorted_order.append(distances.index(min(distances_temp)))
                break
            else:
                # print('Removed ' + str(min(distances_temp)))
                distances_temp.remove(min(distances_temp))
        # print(str(other_track) + ' ' + str(distances))
        # print('end' + str(distances))
        # sorted()

    playlist_data_full['order'] = sorted_order
    playlist_data_full = playlist_data_full.sort_values(by=['order'])
    playlist_data_full['order'] = np.arange(1, playlist_data_full.shape[0]+1) # convert order to integer playlist track number
        
    # Evaluate new order
    sorted_order_eval = evaluate_data(playlist_data_full)
    sorted_evals_next.append(sorted_order_eval)

    # y_true = playlist_data_full['track_number']
    # y_pred = playlist_data_full['order']
    # mean_squared_errors_random.append(mean_squared_error(y_true, y_pred))

    track_eval_changes = []

    for metric in range(len(original_order_eval)):
        track_eval_changes.append(sorted_order_eval[metric] - original_order_eval[metric])
        
    eval_changes_next.append(track_eval_changes)

In [19]:
# print(np.mean(mean_squared_errors), np.mean(mean_squared_errors_next))

evaluation_vector_titles = ['keyTransition', 'tempoTransition', 'trackNumTransition', 'loudnessTransition', 'timbreTransition',
                            'danceabilityTransition', 'energyTransition', 'speechinessTransition','acousticnessTransition',
                            'instrumentalnessTransition', 'livenessTransition', 'valenceTransition']
# original_evals_df = pd.DataFrame.from_records(original_evals, columns=evaluation_vector_titles)
# sorted_evals_df = pd.DataFrame.from_records(sorted_evals, columns=evaluation_vector_titles)
# eval_changes_df = pd.DataFrame.from_records(eval_changes, columns=evaluation_vector_titles)
original_evals_next_df = pd.DataFrame.from_records(original_evals_next, columns=evaluation_vector_titles)
sorted_evals_next_df = pd.DataFrame.from_records(sorted_evals_next, columns=evaluation_vector_titles)
eval_changes_next_df = pd.DataFrame.from_records(eval_changes_next, columns=evaluation_vector_titles)

original_evals_next_df.drop(columns = ['trackNumTransition', 'loudnessTransition'], axis = 1, inplace = True)
sorted_evals_next_df.drop(columns = ['trackNumTransition', 'loudnessTransition'], axis = 1, inplace = True)
eval_changes_next_df.drop(columns = ['trackNumTransition', 'loudnessTransition'], axis = 1, inplace = True)

# original_evals_df['trackTotal'] = original_evals_df.mean(numeric_only=True, axis=1)
# original_evals_df.loc['metricTotalAlbumOriginal'] = original_evals_df.mean(numeric_only=True, axis=0)
# sorted_evals_df['trackTotal'] = sorted_evals_df.mean(numeric_only=True, axis=1)
# sorted_evals_df.loc['metricTotalAlbumSorted'] = sorted_evals_df.mean(numeric_only=True, axis=0)
# eval_changes_df['trackTotal'] = eval_changes_df.mean(numeric_only=True, axis=1)
# eval_changes_df.loc['metricTotalAlbumChanges'] = eval_changes_df.mean(numeric_only=True, axis=0)
original_evals_next_df['trackTotal'] = original_evals_next_df.mean(numeric_only=True, axis=1)
original_evals_next_df.loc['metricTotalNextOriginal'] = original_evals_next_df.mean(numeric_only=True, axis=0)
sorted_evals_next_df['trackTotal'] = sorted_evals_next_df.mean(numeric_only=True, axis=1)
sorted_evals_next_df.loc['metricTotalNextSorted'] = sorted_evals_next_df.mean(numeric_only=True, axis=0)
eval_changes_next_df['trackTotal'] = eval_changes_next_df.mean(numeric_only=True, axis=1)
eval_changes_next_df.loc['metricTotalNextChanges'] = eval_changes_next_df.mean(numeric_only=True, axis=0)

evals_df = pd.DataFrame([#original_evals_df.loc['metricTotalAlbumOriginal'],
                        # sorted_evals_df.loc['metricTotalAlbumSorted'],
                        # eval_changes_df.loc['metricTotalAlbumChanges'],
                        original_evals_next_df.loc['metricTotalNextOriginal'],
                        sorted_evals_next_df.loc['metricTotalNextSorted'],
                        eval_changes_next_df.loc['metricTotalNextChanges']])



evals_df

,keyTransition,tempoTransition,timbreTransition,danceabilityTransition,energyTransition,speechinessTransition,acousticnessTransition,instrumentalnessTransition,livenessTransition,valenceTransition,trackTotal
metricTotalNextOriginal,0.291990,0.337591,0.732499,0.219139,0.337444,0.084244,0.059226,0.261687,0.055702,0.232294,0.261182
metricTotalNextSorted,0.255873,0.266209,0.662262,0.240868,0.351069,0.121506,0.026398,0.234291,0.062133,0.193123,0.241373
metricTotalNextChanges,-0.036117,-0.071382,-0.070237,0.021729,0.013625,0.037262,-0.032828,-0.027396,0.006431,-0.039171,-0.019809
